In [104]:
import pandas as pd
import plotly.graph_objects as go
import openpyxl as xl

In [9]:
import requests
import time
import pandas as pd
import numpy as np

def Convert(dfs,stock):
    names = []
    names_values = []
    basic_info = []
    try:
        df = dfs[5]
        basic_df = dfs[4]
        for i in range(0,len(df.columns)):
            if i%2 == 0:
                #display(df[i])
                names.append(df[i])
            else:
                names_values.append(df[i])
                
        Index_names = pd.concat(names).reset_index(drop= True)
        Index_Values = pd.concat(names_values).reset_index(drop= True)
        df1 = pd.DataFrame()
        df1[f'{stock}'] = Index_names
        df1['Values'] = Index_Values
        #df2 = df1['Values'].replace('%','')
        df2 = df1['Values'].replace('-','0')
        #df1 = df1.fillna('-1')
        
        #Adding stock basic info
        basic_df = basic_df.T.drop(0,axis = 1)
        basic_df = basic_df.rename({1:'Company',2:'Sector'},axis=1)
        basic_df.index = [stock]

        #Adds Percentages to name
        not_add_percentage = df1[~df1['Values'].str.contains('%')]
        not_add_percentage  = not_add_percentage[f'{stock}']
        add_percentage = df1[df1['Values'].str.contains('%')]
        add_percentage  = add_percentage[f'{stock}'] + ' %'

        #Converts that column so I can remove %
        df1['Values'] = df1['Values'].convert_dtypes(str)
        df1['Values'] = df1['Values'].str.replace('%','')

        df1['Values'] = df1['Values'].str.replace('K','e3')
        df1['Values'] = df1['Values'].str.replace('M','e6')
        df1['Values'] = df1['Values'].str.replace('B','e9')

        df1['Index'] = pd.concat([not_add_percentage,add_percentage]).sort_index()
        #df1.index = df1['Index']
        print(df1.index)
        df1 = df1.drop([f'{stock}','Index'], axis=1)
        print(df1)
        df1.columns = [f'{stock}']
        print(df1.columns)
        df2  = df1.T
        print(df2.index)
    
        #display(df2)
        for i in df2.columns: #converts everything to float
            try:
                df2['{}'.format(i)] = df2['{}'.format(i)].astype(float)
            except:
                #print(i)
                0
        df3 =  basic_df.join(df2, how='outer')
        #display(df2.T,basic_df,df3)
        
        return df3
    except:
        print(f'2-{stock}')
        
  
saved = []
stocks = pd.read_csv(r'C:\Users\Tafimul\Python Main Projects\Dashboard\Stock_Screener\SAVE\ETFs.csv')
print(len(stocks))
stocks = stocks.drop_duplicates('Ticker')
print(len(stocks))
#stocks = pd.read_csv('SAVE\ETFs.csv')
start = time.time()
for stock in stocks['Ticker'][0:2]:
  try:
    url = f'https://finviz.com/quote.ashx?t={stock}'

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    dfs = pd.read_html(r.text)
    #print(dfs)
    d = Convert(dfs,stock)
    #saved.append(d)
  except:
    print('1')
print(saved) 
#savethisdf = pd.concat(saved)
savethisdf.index.names = ['Tickers']
savethisdf[['Sector','Industry','Country']] = savethisdf['Sector'].str.split('|',expand=True)
savethisdf = savethisdf[savethisdf.columns[0:74]]
savethisdf = savethisdf.replace('-',np.nan)
savethisdf.to_csv('Dashboard\Stock_Screener\SAVE\FUND.csv')
end = time.time()
print(end - start)
  #return saved
#getFinData()

671
671
RangeIndex(start=0, stop=72, step=1)
            AA     Values         Index
0        Index          -         Index
1   Market Cap     6.49e9    Market Cap
2       Income   963.00e6        Income
3        Sales    13.39e9         Sales
4      Book/sh      31.13       Book/sh
..         ...        ...           ...
67         ATR       2.93           ATR
68  Volatility  7.00 5.88  Volatility %
69  Prev Close      37.52    Prev Close
70       Price      35.51         Price
71      Change      -5.36      Change %

[72 rows x 3 columns]
2-AA
RangeIndex(start=0, stop=72, step=1)
           AAL      Values         Index
0        Index     S&P 500         Index
1   Market Cap      8.23e9    Market Cap
2       Income  -1922.00e6        Income
3        Sales     40.72e9         Sales
4      Book/sh      -12.95       Book/sh
..         ...         ...           ...
67         ATR        0.63           ATR
68  Volatility   5.40 4.18  Volatility %
69  Prev Close       12.71    Prev Close


ValueError: Columns must be same length as key

In [5]:
saved

671

In [105]:
wb = xl.load_workbook('Networth.xlsx',data_only=True) # data only so it doesnt read the formulas
#print(wb.sheetnames)
ws = wb['Summary Table'] #This is the only thing we care about 
df = pd.DataFrame(ws.values) #converts it into a df 

#Organizes the dataframe
df = df.set_index(df[0])
df = df[df.columns[2:]].T
df = df.set_index('Date')

In [108]:
fig = go.Figure()
df1 = df.drop(columns= ['Assets','Liabilities'],axis = 0) # Drops the columns you don't need for the graph

#To make the bars 
for col in df1[:-1].columns:
    if col == 'Net Worth': 
        fig.add_trace(go.Scatter(x = df.index,y = df[f'{col}'], name = col)) # Adds the Networth line
    else:
        fig.add_bar(x = df.index,y = df[f'{col}'], name = col) 
        
#Updates the figure 
fig.update_xaxes(title = 'Time') 
fig.update_yaxes(title = 'Cash') 
fig.update_layout(barmode='relative', title_text='Networth Tracker')
print(df)

0           Assets Cash / Cash Equivalents Investments Retirement Investments  \
Date                                                                            
2022-01-01  372681                    4700       57939                   4942   
2022-02-01  359736                    1713     43546.4                5176.26   
2022-03-01  361090                    1500       47882                5707.97   
2022-04-01  380369                    4259       54675                 6434.7   
2022-05-01  383231                    4500     45027.3                6703.22   
2022-06-01  380718                    9935     32649.5                7524.86   
2022-07-01  417595                   24368     22667.8                 7558.8   
2022-08-01       0                       0           0                      0   
2022-09-01       0                       0           0                      0   
2022-10-01       0                       0           0                      0   
2022-11-01       0          

In [107]:
df.drop(columns= ['Assets','Liabilities'],axis = 0)

,Cash / Cash Equivalents,Investments,Retirement Investments,Physical Assets,ST Liabilities,LT Liabilities,Net Worth
Date,,,,,,,
2022-01-01,4700,57939,4942,305100,-5000,-270888,96793
2022-02-01,1713,43546.4,5176.26,309300,-3700,-269676,86359
2022-03-01,1500,47882,5707.97,306000,-3500,-268937,88653
2022-04-01,4259,54675,6434.7,315000,-3500,-268556,108312
2022-05-01,4500,45027.3,6703.22,327000,-6000,-267460,109770
2022-06-01,9935,32649.5,7524.86,330609,-9397.76,-266977,104343
2022-07-01,24368,22667.8,7558.8,363000,-8500,-265746,143348
2022-08-01,0,0,0,0,0,-235973,-235973
2022-09-01,0,0,0,0,0,-235592,-235592


In [4]:
import pandas as pd
df = pd.read_csv('Stock Screener\SAVE\Stocks Table.csv')
df.columns
df['Sector'].unique()
df['Industry'].unique()

Index(['Ticker', 'Price', '52-Week-Low', '52-Week-High', '20-MA', '50-MA',
       '100-MA', '150-MA', '200-MA', 'Sector', 'Industry', 'P/E', 'PEG',
       'P/FCF', 'Sales Q/Q %', 'EPS this Y %', 'EPS next Y %', 'Death MACD',
       'Golden MACD', 'MTUM-MA-25', 'MTUM-EMA-25', 'Weekly Death MACD',
       'Weekly Golden MACD'],
      dtype='object')